# ***Precio Promedio por M2 cuadrados***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre or 'df_junio_2023_queretaro' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [2]:
df_junio_2023_queretaro.columns

Index(['id', 'q', 'categoria', 'ids', 'propiedad', 'precio', 'm2_total',
       'm2_construido', 'precio_m2_terreno', 'precio_m2_cons',
       'publicado_hace', 'personas_interesadas', 'promedio_interes_mensual',
       'status', 'tipo', 'estacionamiento', 'recamaras', 'baños', 'medio_baño',
       'baño_total', 'antiguedad', 'seguridad_privada', 'fraccionamiento',
       'colonia', 'cp', 'url'],
      dtype='object')

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio','metros_total':'m2_total','precio_m2_cons':'precio_m2_construido','metros_construido':'m2_construido','precio_m2_contruido':'precio_m2_construido','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['precio', 'm2_total','m2_construido', 'precio_m2_construido', 'precio_m2_terreno']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

### *DF CON RANGOS DE M2 Y SU PRECIO*

In [5]:
new_dataframes_list = []

# Definir los límites de los rangos
rangos = [0, 100, 150, 200, 250, 300, float('inf')]
labels = ['[0-100]', '[100-150]', '[150-200]', '[200-250]', '[250-300]', '[> 300]']

# Iterar sobre cada DataFrame en la lista original
for df in dataframes_list:
    # Crear una copia del DataFrame
    modified_df = df.copy()
    # Asignar el rango correspondiente a cada valor de 'm2_construido'
    modified_df['rango'] = pd.cut(modified_df['m2_total'], bins=rangos, labels=labels, right=False)
    # Crear un nuevo DataFrame solo con las columnas 'rango' y 'precio_m2_terreno'
    new_df = modified_df[['rango', 'precio_m2_terreno']].copy()
    # Agregar el nuevo DataFrame a la lista
    new_dataframes_list.append(new_df)

# Imprimir los primeros registros de cada nuevo DataFrame
for idx, df in enumerate(new_dataframes_list):
    print(f"Primeros registros de nuevo DataFrame {idx + 1}:")
    print(df.head())
    print()  # Agregar una línea en blanco entre cada DataFrame


Primeros registros de nuevo DataFrame 1:
       rango  precio_m2_terreno
0  [150-200]           16500.00
1    [> 300]           16238.32
2  [100-150]           20629.66
3    [0-100]            4590.16
4    [> 300]            1311.48

Primeros registros de nuevo DataFrame 2:
     rango  precio_m2_terreno
0  [0-100]            4590.16
1  [0-100]            3625.00
2  [0-100]            4373.33
3  [0-100]            4206.60
4  [0-100]            4933.83

Primeros registros de nuevo DataFrame 3:
       rango  precio_m2_terreno
0    [0-100]            5970.15
1    [0-100]            8809.00
2  [150-200]            2605.56
3    [0-100]           11904.76
4    [0-100]            8214.29

Primeros registros de nuevo DataFrame 4:
       rango  precio_m2_terreno
0  [150-200]           16500.00
1    [> 300]           16238.32
2  [100-150]           20629.66
3    [0-100]            4590.16
4    [> 300]            1311.48

Primeros registros de nuevo DataFrame 5:
     rango  precio_m2_terreno
0  [0

### *Promedio de precio_m2_terreno por rango CADA DF* 

In [6]:
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    print(f"Promedio de precio_m2_terreno por rango en el DataFrame {idx + 1}:")
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()
    print(promedios_por_rango)
    print()  # Agregar una línea en blanco entre cada resultado

Promedio de precio_m2_terreno por rango en el DataFrame 1:
rango
[0-100]      25869.767270
[100-150]    24773.376092
[150-200]    24358.928155
[200-250]    23333.513686
[250-300]    23682.210228
[> 300]      20836.457875
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 2:
rango
[0-100]      24485.873675
[100-150]    50075.857737
[150-200]    24253.072751
[200-250]    22611.840909
[250-300]    22844.675565
[> 300]      26030.852122
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 3:
rango
[0-100]      32642.154546
[100-150]    49312.286545
[150-200]    25563.608424
[200-250]    24786.736934
[250-300]    23824.182019
[> 300]      21957.494704
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 4:
rango
[0-100]      23717.964323
[100-150]    23851.481886
[150-200]    23588.385923
[200-250]    23097.725778
[250-300]    23601.582687
[> 300]      20751.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5820\2294499642.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()


In [7]:
# Inicializar un diccionario para almacenar los promedios de precio por metro cuadrado para cada rango
promedios_por_rango_dict = {}
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()
    # Actualizar el diccionario con los promedios calculados
    promedios_por_rango_dict[f'DataFrame_{idx + 1}'] = promedios_por_rango

# Convertir el diccionario en un DataFrame
rango_precio_m2 = pd.DataFrame(promedios_por_rango_dict)
print("DataFrame 'rango_precio_m2':")
rango_precio_m2

DataFrame 'rango_precio_m2':


C:\Users\yoe11\AppData\Local\Temp\ipykernel_5820\225855980.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()


,DataFrame_1,DataFrame_2,DataFrame_3,DataFrame_4,DataFrame_5
rango,,,,,
[0-100],25869.767270,24485.873675,32642.154546,23717.964323,25110.693133
[100-150],24773.376092,50075.857737,49312.286545,23851.481886,23138.834023
[150-200],24358.928155,24253.072751,25563.608424,23588.385923,22583.406820
[200-250],23333.513686,22611.840909,24786.736934,23097.725778,20692.491206
[250-300],23682.210228,22844.675565,23824.182019,23601.582687,20401.423225
[> 300],20836.457875,26030.852122,21957.494704,20751.791012,17605.746130


In [8]:
# Calcular el promedio general de cada rango en todas las columnas de rango_precio_m2
promedio_general_por_rango = rango_precio_m2.mean(axis=1)
# Crear un nuevo DataFrame 'Precio_mean_M2' con los rangos y sus promedios generales
Precio_mean_M2 = pd.DataFrame({'Rango': promedio_general_por_rango.index, 'Precio_m2_mean': promedio_general_por_rango.values})

print("DataFrame 'Precio_mean_M2':")
Precio_mean_M2

DataFrame 'Precio_mean_M2':


,Rango,Precio_m2_mean
0,[0-100],26365.290589
1,[100-150],34230.367257
2,[150-200],24069.480414
3,[200-250],22904.461703
4,[250-300],22870.814745
5,[> 300],21436.468369


# Gráfica

In [9]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']
porcentajes = Precio_mean_M2['Precio_m2_mean']

# Convertir la columna 'Precio_m2_mean' a números de punto flotante
#Precio_mean_M2['Precio_m2_mean'] = Precio_mean_M2['Precio_m2_mean'].str.replace('$', '').str.replace(',', '').astype(float)
# Convertir los valores a cadenas con el formato deseado
textos_formateados = [f'${round(valor, 2):,.2f}' for valor in Precio_mean_M2['Precio_m2_mean']]

fig = go.Figure()
# Obtener el valor máximo redondeado al próximo múltiplo de 5000
max_valor = int(Precio_mean_M2['Precio_m2_mean'].max() / 5000 + 1) * 5000

# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=Precio_mean_M2['Rango'],  # Eje x: Modalidades
    y=Precio_mean_M2['Precio_m2_mean'],  # Eje y: Porcentajes
    marker_color=colores,  # Especifica los colores de las barras
    text=textos_formateados,  # Texto que se mostrará en las barras (porcentaje)
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Precio promedio por M2 totales.',  # Título de la gráfica
    #xaxis=dict(title='Rango'),  # Título del eje x
    yaxis=dict(
        #title='Precio', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
        tickvals=list(range(0, max_valor + 5000, 5000)),  # Valores de los ticks del eje y
        ticktext=[f'${valor:.2f}' for valor in range(0, max_valor + 5000, 5000)]  # Textos de los ticks del eje y
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    height=500,  # Altura de la gráfica
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)

)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_PrecioPromedio_m2', carpeta='assets/graficas')
fig.show()

## POR M2_CONSTRUCCIÓN

In [10]:
new_dataframes_list = []

# Definir los límites de los rangos
rangos = [0, 100, 150, 200, 250, 300, float('inf')]
labels = ['[0-100]', '[100-150]', '[150-200]', '[200-250]', '[250-300]', '[> 300]']

# Iterar sobre cada DataFrame en la lista original
for df in dataframes_list:
    # Crear una copia del DataFrame
    modified_df = df.copy()
    # Asignar el rango correspondiente a cada valor de 'm2_construido'
    modified_df['rango'] = pd.cut(modified_df['m2_total'], bins=rangos, labels=labels, right=False)
    # Crear un nuevo DataFrame solo con las columnas 'rango' y 'precio_m2_terreno'
    new_df = modified_df[['rango', 'precio_m2_construido']].copy()
    # Agregar el nuevo DataFrame a la lista
    new_dataframes_list.append(new_df)

# Imprimir los primeros registros de cada nuevo DataFrame
for idx, df in enumerate(new_dataframes_list):
    print(f"Primeros registros de nuevo DataFrame {idx + 1}:")
    print(df.head())
    print()  # Agregar una línea en blanco entre cada DataFrame


Primeros registros de nuevo DataFrame 1:
       rango  precio_m2_construido
0  [150-200]              17254.90
1    [> 300]              20746.27
2  [100-150]              18410.01
3    [0-100]               4590.16
4    [> 300]               1538.46

Primeros registros de nuevo DataFrame 2:
     rango  precio_m2_construido
0  [0-100]               4590.16
1  [0-100]               3625.00
2  [0-100]               3644.44
3  [0-100]               3868.14
4  [0-100]               4933.83

Primeros registros de nuevo DataFrame 3:
       rango  precio_m2_construido
0    [0-100]               5970.15
1    [0-100]               8809.00
2  [150-200]               2605.56
3    [0-100]              11904.76
4    [0-100]               8214.29

Primeros registros de nuevo DataFrame 4:
       rango  precio_m2_construido
0  [150-200]              17254.90
1    [> 300]              20746.27
2  [100-150]              18410.01
3    [0-100]               4590.16
4    [> 300]               1538.46

Prim

### *Promedio de precio_m2_terreno por rango CADA DF* 

In [11]:
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    print(f"Promedio de precio_m2_construido por rango en el DataFrame {idx + 1}:")
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_construido'].mean()
    print(promedios_por_rango)
    print()  # Agregar una línea en blanco entre cada resultado

Promedio de precio_m2_construido por rango en el DataFrame 1:
rango
[0-100]      24449.840053
[100-150]    22380.385529
[150-200]    22566.185688
[200-250]    21922.715236
[250-300]    22942.990293
[> 300]      25093.035895
Name: precio_m2_construido, dtype: float64

Promedio de precio_m2_construido por rango en el DataFrame 2:
rango
[0-100]      23250.718559
[100-150]    46674.157737
[150-200]    21381.786457
[200-250]    20590.417273
[250-300]    20674.820615
[> 300]      37697.094249
Name: precio_m2_construido, dtype: float64

Promedio de precio_m2_construido por rango en el DataFrame 3:
rango
[0-100]      49958.032455
[100-150]    46211.329928
[150-200]    21273.308573
[200-250]    21656.117279
[250-300]    20766.546711
[> 300]      26227.401505
Name: precio_m2_construido, dtype: float64

Promedio de precio_m2_construido por rango en el DataFrame 4:
rango
[0-100]      22762.604055
[100-150]    21458.766161
[150-200]    21578.859596
[200-250]    20981.320627
[250-300]    21567.71357

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5820\2302470058.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [12]:
# Inicializar un diccionario para almacenar los promedios de precio por metro cuadrado para cada rango
promedios_por_rango_dict = {}
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_construido'].mean()
    # Actualizar el diccionario con los promedios calculados
    promedios_por_rango_dict[f'DataFrame_{idx + 1}'] = promedios_por_rango

# Convertir el diccionario en un DataFrame
rango_precio_m2 = pd.DataFrame(promedios_por_rango_dict)
print("DataFrame 'rango_precio_m2':")
rango_precio_m2

DataFrame 'rango_precio_m2':


C:\Users\yoe11\AppData\Local\Temp\ipykernel_5820\905179478.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,DataFrame_1,DataFrame_2,DataFrame_3,DataFrame_4,DataFrame_5
rango,,,,,
[0-100],24449.840053,23250.718559,49958.032455,22762.604055,20444.387923
[100-150],22380.385529,46674.157737,46211.329928,21458.766161,19899.051731
[150-200],22566.185688,21381.786457,21273.308573,21578.859596,19286.704237
[200-250],21922.715236,20590.417273,21656.117279,20981.320627,19130.003857
[250-300],22942.990293,20674.820615,20766.546711,21567.713579,20295.422485
[> 300],25093.035895,37697.094249,26227.401505,26454.773215,22412.962211


In [13]:
# Calcular el promedio general de cada rango en todas las columnas de rango_precio_m2
promedio_general_por_rango = rango_precio_m2.mean(axis=1)
# Crear un nuevo DataFrame 'Precio_mean_M2' con los rangos y sus promedios generales
Precio_mean_M2 = pd.DataFrame({'Rango': promedio_general_por_rango.index, 'Precio_m2_mean': promedio_general_por_rango.values})

print("DataFrame 'Precio_mean_M2':")
Precio_mean_M2

DataFrame 'Precio_mean_M2':


,Rango,Precio_m2_mean
0,[0-100],28173.116609
1,[100-150],31324.738217
2,[150-200],21217.368910
3,[200-250],20856.114854
4,[250-300],21249.498737
5,[> 300],27577.053415


In [14]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']
porcentajes = Precio_mean_M2['Precio_m2_mean']

# Convertir la columna 'Precio_m2_mean' a números de punto flotante
#Precio_mean_M2['Precio_m2_mean'] = Precio_mean_M2['Precio_m2_mean'].str.replace('$', '').str.replace(',', '').astype(float)
# Convertir los valores a cadenas con el formato deseado
textos_formateados = [f'${round(valor, 2):,.2f}' for valor in Precio_mean_M2['Precio_m2_mean']]

fig = go.Figure()
# Obtener el valor máximo redondeado al próximo múltiplo de 5000
max_valor = int(Precio_mean_M2['Precio_m2_mean'].max() / 5000 + 1) * 5000

# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=Precio_mean_M2['Rango'],  # Eje x: Modalidades
    y=Precio_mean_M2['Precio_m2_mean'],  # Eje y: Porcentajes
    marker_color=colores,  # Especifica los colores de las barras
    text=textos_formateados,  # Texto que se mostrará en las barras (porcentaje)
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Precio promedio por M2 totales.',  # Título de la gráfica
    xaxis=dict(title='M² Construidos'),  # Título del eje x
    yaxis=dict(
        title='Precio promedio', # Título del eje y        
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
        tickvals=list(range(0, max_valor + 5000, 5000)),  # Valores de los ticks del eje y
        ticktext=[f'${valor:.2f}' for valor in range(0, max_valor + 5000, 5000)]  # Textos de los ticks del eje y
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    height=500,  # Altura de la gráfica
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)

)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_bar_PrecioPromedio_m2', carpeta='assets/graficas')
fig.show()
